In [1]:
import os
import sys
sys.path.append('../../src')
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
from datetime import datetime
import time
# import config

import data_process.neg_sample as ng_sample
# import evaluate_entity
from evaluate_entity import CustomHR, CustomNDCG, CustomRoc, CustomRoctop, CustomRecall_top, CustomPrecision_top
from model_entity import EntityCat
from data_utils import CatData
from utils.constants import DEFAULT_USER_COL,DEFAULT_ITEM_COL,DEFAULT_RATING_COL, DEFAULT_TIMESTAMP_COL

from sklearn import metrics, preprocessing
# from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score, recall_score, precision_score

import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

# import argparse
torch.manual_seed(0)

/Users/hao/.pyenv/versions/3.6.15/envs/torch-cpu5/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from ignite.engine import Engine, Events, create_supervised_trainer, create_supervised_evaluator, RemovableEventHandle
from ignite.metrics import Accuracy, Loss, Metric
from ignite.handlers import ModelCheckpoint, EarlyStopping
from ignite.exceptions import NotComputableError
from ignite.metrics.metric import sync_all_reduce, reinit__is_reduced
from ignite.contrib.handlers.tqdm_logger import ProgressBar
# from ignite.contrib.handlers import TensorboardLogger 
from ignite.contrib.handlers.wandb_logger import *

In [6]:
BATCH_SIZE = 20
EPOCHS  = 200
TOP_K = 10
NEG_TEST = 100
NEG_TRAIN = 4

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [8]:
device

device(type='cpu')

In [9]:
df_train1  = ng_sample.read_feather("../../data/jobs/leave_one_train.csv")
df_train2 = pd.read_feather("../../data/jobs/leave_one_train_neg")
df_test_ori = pd.read_feather("../../data/jobs/test_pos_neg").iloc[:101,]
df_all_features = pd.read_csv('../../data/jobs/merged_sub_clean.csv')
df_train1 = df_train1.sort_values(by=[DEFAULT_USER_COL]).iloc[:20,].reset_index(drop=True)
df_train2 = df_train2.sort_values(by=[DEFAULT_USER_COL]).iloc[:20*NEG_TRAIN,].reset_index(drop=True)

In [10]:
df_train1[DEFAULT_RATING_COL] = 1

In [11]:
def concat_index(df1, df2, NEG_TRAIN):
    df2.index = df2.index//NEG_TRAIN
    return pd.concat([df1, df2], axis=0).sort_index(kind='mregesort').reset_index(drop=True)

In [12]:
df_train_all = concat_index(df_train1, df_train2, NEG_TRAIN)

In [13]:
# df_train1  = ng_sample.read_feather("../../data/jobs/leave_one_train.csv")
# df_train2 = pd.read_feather("../../data/jobs/leave_one_train_neg")
# df_test_ori = pd.read_feather("../../data/jobs/test_pos_neg")
# df_all_features = pd.read_csv('../../data/jobs/merged_sub_clean.csv')

In [14]:
df_train_all['flag'] = 1
df_test_ori['flag'] = 0
df_all = pd.concat([df_train_all, df_test_ori], axis=0).reset_index(drop=True)

user features: 
       'WindowID_user', 'Split', 'City',
       'State', 'Country', 'Zip_user', 'DegreeType', 'Major', 'GraduationDate',
       'WorkHistoryCount', 'TotalYearsExperience', 'CurrentlyEmployed',
       'ManagedOthers', 'ManagedHowMany',
       
job features: 
       'WindowID_job', 'City_job',
       'State_job', 'Country_job', 'Zip_job', 'StartDate', 'EndDate',

### Choose the features for the model

In [15]:
user_features = ['City']
user_features_extend = [DEFAULT_USER_COL] + user_features

item_features = ['City_job']
item_features_extend =[DEFAULT_ITEM_COL] + item_features

base_features = [DEFAULT_USER_COL, DEFAULT_ITEM_COL, DEFAULT_RATING_COL, ]

In [16]:
def unique_filter(df_data, name_col):
    df_uni=df_data[~(df_data[name_col].duplicated())].reset_index(drop=True)
    return df_uni

In [17]:
def feature_merge(list_f, df_all, df_all_f, default_x_col):
    df_x_features = df_all_f[list_f]
    df_x_unique = unique_filter(df_x_features, default_x_col)
    df_merge_x = df_all.merge(df_x_unique, how='left', on=[default_x_col])
    return df_merge_x

In [18]:
def user_item_merge(mode, df_all, df_all_features, feature_list):
    if mode =='user':
        df_merge_x = feature_merge(feature_list, df_all, df_all_features,\
                                      DEFAULT_USER_COL)
    if mode == 'item':
        df_merge_x = feature_merge(feature_list, df_all, df_all_features, \
                                   DEFAULT_ITEM_COL)

    return df_merge_x

In [19]:
def mix_merge(df_all, df_all_features, f_list_user, f_list_item):
    df_merge_user = feature_merge(f_list_user, df_all, df_all_features, \
                                  DEFAULT_USER_COL)
    df_merge_item = feature_merge(f_list_item, df_all, df_all_features, \
                                  DEFAULT_ITEM_COL)
    df_merge_x = pd.concat([df_merge_user[f_list_user], \
                             df_merge_item[f_list_item]], axis=1)
    df_merge_x[DEFAULT_RATING_COL] = df_all[DEFAULT_RATING_COL]
    assert df_all.shape[0] == df_merge_x.shape[0], "wrong merge"
    return df_merge_x

In [20]:
df_mix_merge = mix_merge(df_all, df_all_features, user_features_extend, item_features_extend)

In [21]:
le = preprocessing.LabelEncoder()

In [22]:
def cat_encode(df_data, list_f):
    for f in list_f:
        df_data[f] = le.fit_transform(df_data[f].astype('category').cat.codes.values)
    return df_data

In [23]:
features_to_code = df_mix_merge.columns

In [24]:
df_all_encode = cat_encode(df_mix_merge, features_to_code)

In [25]:
# df_all_encode[DEFAULT_RATING_COL] = df_all[DEFAULT_RATING_COL]

In [26]:
df_train = df_all_encode[df_all.flag==1]
df_test = df_all_encode[df_all.flag==0]

# df_train=df_train.drop(['flag'], axis=1)
# df_test=df_test.drop(['flag'], axis=1)

In [27]:
features_to_train = [DEFAULT_USER_COL, DEFAULT_ITEM_COL]+ user_features + item_features +[DEFAULT_RATING_COL]
df_train = df_train[features_to_train]
df_test = df_test[features_to_train]

In [28]:
tb_cf = "-".join(user_features)+'-'.join(item_features)
tb_cf

'CityCity_job'

In [29]:
df_train=df_train[features_to_train]
df_test=df_test[features_to_train]

num_feature=[]
features_to_train.remove(DEFAULT_RATING_COL)

In [30]:
def data_split_user(df_train, val_size=0.2):
    unique_user = df_train[DEFAULT_USER_COL].unique()
    val_user = np.random.choice(unique_user, int(val_size*len(unique_user)), replace=False)
    df_train_split = df_train[~(df_train[DEFAULT_USER_COL].isin(val_user))]
    df_val_split = df_train[df_train[DEFAULT_USER_COL].isin(val_user)]
    return df_train_split, df_val_split

In [31]:
df_train_split, df_val_split = data_split_user(df_train, val_size=0.5)

In [32]:
np_train = df_train_split.values
np_val = df_val_split.values
np_test = df_test.values

In [33]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [34]:
train_dataset = CatData(np_train)
val_dataset = CatData(np_val)
test_dataset = CatData(np_test) 
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0,  worker_init_fn=seed_worker,generator=g)
val_loader = data.DataLoader(val_dataset, batch_size=NEG_TRAIN+1, shuffle=False, num_workers=0,  worker_init_fn=seed_worker,generator=g)
test_loader = data.DataLoader(test_dataset, batch_size=NEG_TEST+1, shuffle=False, num_workers=0,worker_init_fn=seed_worker,generator=g )

In [35]:
embedding_size = []
for c in features_to_train:
    num_unique_values = int(df_all_encode[c].nunique())
    embed_dim = int(min(np.ceil(num_unique_values/2), 50))
    embedding_size.append([num_unique_values, embed_dim])  

In [36]:
model = EntityCat(embedding_size = embedding_size, num_numerical_cols = len(num_feature),
               output_size = 2)
model.to(device)

EntityCat(
  (all_embeddings): ModuleList(
    (0): Embedding(8, 4)
    (1): Embedding(160, 50)
    (2): Embedding(8, 4)
    (3): Embedding(116, 50)
  )
  (embedding_dropout): Dropout(p=0.4, inplace=False)
  (layers): Sequential(
    (0): Linear(in_features=108, out_features=100, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.4, inplace=False)
    (3): Linear(in_features=100, out_features=2, bias=True)
  )
)

In [37]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [38]:
print(model)

EntityCat(
  (all_embeddings): ModuleList(
    (0): Embedding(8, 4)
    (1): Embedding(160, 50)
    (2): Embedding(8, 4)
    (3): Embedding(116, 50)
  )
  (embedding_dropout): Dropout(p=0.4, inplace=False)
  (layers): Sequential(
    (0): Linear(in_features=108, out_features=100, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.4, inplace=False)
    (3): Linear(in_features=100, out_features=2, bias=True)
  )
)


In [39]:
def output_trans_loss(output):
    return output['y_pred'], output['label']

val_metrics = {
#     "customEva": CustomEva(topk=3, threshold=0.5),
    'hr': CustomHR(),
    'ndcg': CustomNDCG(),
    'roc': CustomRoc(),
    'roc_top': CustomRoctop(),
    'recall_top': CustomRecall_top(threshold=0.5),
    'precision_top': CustomPrecision_top(threshold=0.5),
    "loss": Loss(criterion, output_transform=output_trans_loss)
}

In [56]:
def train_step(engine, batch):
    model.train()
    optimizer.zero_grad()
    x, y = batch[0].to(device), batch[1].to(device)
    y_pred = model(x)[:,1]
    loss = criterion(y_pred, y.float())
    loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_step)

def validation_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, label = batch[0].to(device), batch[1].to(device)
        y_pred = model(x)[:,1]
        label=label.float()
        
        y_pred_top, indices = torch.topk(y_pred, engine.state.topk)
        y_pred_top = y_pred_top.detach().cpu().numpy()
        reco_item = torch.take(x[:,1], indices).cpu().numpy().tolist()
        pos_item = x[0,1].cpu().numpy().tolist()  # ground truth, item id
        label_top = label[indices].cpu().numpy()
        
        return {'pos_item':pos_item, 'reco_item':reco_item, 'y_pred_top':y_pred_top, 'label_top':label_top, 'label':label, 'y_pred':y_pred}
#         return {'pos_item':pos_item, 'reco_item':reco_item,'label':label,'y_pred':y_pred }

train_evaluator = Engine(validation_step)
train_evaluator.state_dict_user_keys.append('topk')

val_evaluator = Engine(validation_step)
val_evaluator.state_dict_user_keys.append('topk')

@val_evaluator.on(Events.STARTED)
def init_user_value():
    val_evaluator.state.topk=3
    
@train_evaluator.on(Events.STARTED)
def init_user_value():
    train_evaluator.state.topk=3
    
    
# Attach metrics to the evaluators
for name, metric in val_metrics.items():
    metric.attach(train_evaluator, name)

for name, metric in val_metrics.items():
    metric.attach(val_evaluator, name)

# Eearly_stop 
def score_function(engine):
    val_loss = engine.state.metrics['roc']
    return val_loss

Eearly_stop_handler = EarlyStopping(patience=10, score_function=score_function, trainer=trainer)
val_evaluator.add_event_handler(Events.COMPLETED, Eearly_stop_handler)

In [57]:
@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    train_evaluator.run(train_loader)
    metrics = train_evaluator.state.metrics
#     hr, ndcg, roc, roc_top, recall, precision = metrics['customEva']
    hr = metrics['hr']
    ndcg = metrics['ndcg']
    roc = metrics['roc']
    roc_top = metrics['roc_top']
    recall = metrics['recall_top']
    precision = metrics['precision_top']
    loss = metrics['loss']
    print(f"Training Results - Epoch[{trainer.state.epoch}]  Avg loss: {loss:.2f} \
     Avg ndcg: {ndcg:.2f}  Avg roc: {roc:.2f}  Avg roc_top: {roc_top:.2f} \
      Avg recall: {recall:.2f}  Avg precision: {precision:.2f}")


@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    val_evaluator.run(val_loader)
    metrics = val_evaluator.state.metrics
#     hr, ndcg, roc, roc_top, recall, precision = metrics['customEva']
    hr = metrics['hr']
    ndcg = metrics['ndcg']
    roc = metrics['roc']
    roc_top = metrics['roc_top']
    recall = metrics['recall_top']
    precision = metrics['precision_top']
    loss = metrics['loss']
    print(f"Validation Results - Epoch[{trainer.state.epoch}]  Avg loss: {loss:.2f} \
     Avg ndcg: {ndcg:.2f}  Avg roc: {roc:.2f}  Avg roc_top: {roc_top:.2f} \
      Avg recall: {recall:.2f}  Avg precision: {precision:.2f}")

In [59]:
wandb_logger = WandBLogger(
    project="pytorch-ignite-integration",
    name="jrs-reco",
    config={"max_epochs": EPOCHS,"batch_size":BATCH_SIZE},
    tags=["pytorch-ignite", "jrs"]
)

to_save = {'model': model}
checkpoint_handler = ModelCheckpoint(
    wandb_logger.run.dir,
    n_saved=2, filename_prefix='best',
    score_name="roc",
    global_step_transform=global_step_from_engine(trainer)
)
val_evaluator.add_event_handler(Events.COMPLETED, checkpoint_handler, to_save)


'''
Attach the Object to the output handlers:
1) Log training loss - attach to trainer
2) Log validation loss - attach to evaluator
3) Log optional Parameters
4) Watch the model
'''
    

wandb_logger.attach_output_handler(
    trainer,
    event_name=Events.ITERATION_COMPLETED,
    tag="training",
    output_transform=lambda loss: {"loss": loss}
)

wandb_logger.attach_output_handler(
    train_evaluator,
    event_name=Events.EPOCH_COMPLETED,
    tag="training",
    metric_names=['loss',"roc", 'hr', 'ndcg','roc_top', 'precision_top','recall_top'],
#     hr, ndcg, roc, roc_top, recall, precision = metrics['customEva']
    global_step_transform=lambda *_: trainer.state.iteration,
)

wandb_logger.attach_output_handler(
    val_evaluator,
    event_name=Events.EPOCH_COMPLETED,
    tag="validation",
    metric_names=['loss',"roc", 'hr', 'ndcg','roc_top', 'precision_top','recall_top'],
    global_step_transform=lambda *_: trainer.state.iteration,
)

wandb_logger.attach_opt_params_handler(
    trainer,
    event_name=Events.ITERATION_STARTED,
    optimizer=optimizer,
    param_name='lr'  # optional
)

wandb_logger.watch(model)

trainer.run(train_loader, max_epochs=10)

lr/group_0,▁▁▁
training/hr,▁
training/loss,█▁▂
training/ndcg,▁
training/precision_top,▁
training/recall_top,▁
training/roc,▁
training/roc_top,▁
validation/hr,▁
validation/loss,▁
validation/ndcg,▁


Training Results - Epoch[1]  Avg loss: 0.67      Avg ndcg: 0.54  Avg roc: 1.00  Avg roc_top: 0.00       Avg recall: 0.67  Avg precision: 0.67
Validation Results - Epoch[1]  Avg loss: 0.75      Avg ndcg: 0.63  Avg roc: 0.72  Avg roc_top: 0.50       Avg recall: 0.11  Avg precision: 0.11
Training Results - Epoch[2]  Avg loss: 0.67      Avg ndcg: 0.54  Avg roc: 1.00  Avg roc_top: 0.00       Avg recall: 0.67  Avg precision: 0.67
Validation Results - Epoch[2]  Avg loss: 0.74      Avg ndcg: 0.63  Avg roc: 0.72  Avg roc_top: 0.50       Avg recall: 0.11  Avg precision: 0.11
Training Results - Epoch[3]  Avg loss: 0.67      Avg ndcg: 0.54  Avg roc: 1.00  Avg roc_top: 0.00       Avg recall: 0.67  Avg precision: 0.67
Validation Results - Epoch[3]  Avg loss: 0.74      Avg ndcg: 0.63  Avg roc: 0.72  Avg roc_top: 0.50       Avg recall: 0.11  Avg precision: 0.11
Training Results - Epoch[4]  Avg loss: 0.67      Avg ndcg: 0.54  Avg roc: 1.00  Avg roc_top: 0.00       Avg recall: 0.67  Avg precision: 0.67


State:
	iteration: 30
	epoch: 10
	epoch_length: 3
	max_epochs: 10
	output: 0.6684209108352661
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: torch.utils.data.dataloader.DataLoader
	seed: <class 'NoneType'>
	times: <class 'dict'>